In [27]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

In [28]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from __future__ import division
import datetime

from pandas_datareader import data as web

# Get the Data
First we use the yahoo finance API to get 2 years worth of TSLA stock data (2019 to 2021)

In [29]:
#getting the data
def get(tickers, startdate, enddate):
  def data(ticker):
    return (web.DataReader(ticker, data_source='yahoo', start=startdate, end=enddate))
  datas = map (data, tickers)
  return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))

tickers = ['TSLA']
df = get(tickers, datetime.datetime(2019, 1, 1), datetime.datetime(2021, 1, 1))

In [30]:
df

High         Low  ...      Volume   Adj Close
Ticker Date                                ...                        
TSLA   2019-01-02   63.026001   59.759998  ...  58293000.0   62.023998
       2019-01-03   61.880001   59.476002  ...  34826000.0   60.071999
       2019-01-04   63.599998   60.546001  ...  36970500.0   63.537998
       2019-01-07   67.348000   63.549999  ...  37756000.0   66.991997
       2019-01-08   68.802002   65.403999  ...  35042500.0   67.070000
...                       ...         ...  ...         ...         ...
       2020-12-24  666.090027  641.000000  ...  22865600.0  661.770020
       2020-12-28  681.400024  660.799988  ...  32278600.0  663.690002
       2020-12-29  669.900024  655.000000  ...  22910800.0  665.989990
       2020-12-30  696.599976  668.359985  ...  42846000.0  694.780029
       2020-12-31  718.719971  691.119995  ...  49649900.0  705.669983

[505 rows x 6 columns]

# Indicator Functions


1.   Exponential MOving Average
2.   Price Rate of Change (ROC)
3. Relative Strength Index (RSI)
4. Simple Moving Average



In [31]:
# An exponential moving average (EMA) is a type of moving average (MA) that places a greater weight and significance on the most recent data points.
def EMA(df, n):
    EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name='EMA_' + str(n))
    return EMA

In [32]:
#The Price Rate of Change (ROC) is a momentum-based technical indicator that measures the 
# percentage change in price between the current price and the price a certain number of periods ago.

def ROC(df, n):  
    M = df.diff(n - 1)  
    N = df.shift(n - 1)  
    ROC = pd.Series(((M / N) * 100), name = 'ROC_' + str(n))   
    return ROC

In [33]:
#RSI is a momentum oscillator that measures the speed and change of price movements.

def RSI(series, period):
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])

 rs=pd.DataFrame.ewm(u,com=period-1,adjust=False).mean() 
 pd.DataFrame.ewm(d,com=period-1,adjust=False).mean()

 return 100 - 100 / (1 + (rs))

In [34]:
#Moving Average

def MA(df, n):
    MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))
    return MA

Now we calculate the values and all to our DataFrame

In [35]:
df['EMA20'] = EMA(df, 20)
df['ROC20'] = ROC(df['Close'], 20)
df['RSI20'] = RSI(df['Close'], 20)
df['MA20'] = MA(df, 20)

In [36]:
df

High         Low  ...      RSI20        MA20
Ticker Date                                ...                       
TSLA   2019-01-02   63.026001   59.759998  ...        NaN         NaN
       2019-01-03   61.880001   59.476002  ...        NaN         NaN
       2019-01-04   63.599998   60.546001  ...        NaN         NaN
       2019-01-07   67.348000   63.549999  ...        NaN         NaN
       2019-01-08   68.802002   65.403999  ...        NaN         NaN
...                       ...         ...  ...        ...         ...
       2020-12-24  666.090027  641.000000  ...  92.405523  623.862003
       2020-12-28  681.400024  660.799988  ...  92.098041  627.758502
       2020-12-29  669.900024  655.000000  ...  91.794760  632.678003
       2020-12-30  696.599976  668.359985  ...  92.347409  638.179004
       2020-12-31  718.719971  691.119995  ...  92.312774  645.021503

[505 rows x 10 columns]

Since we have used the indicators with a 20 day timeframe, for the first 20 days of the data NaN values would be printed

# Removing the Null values

In [37]:
print(df.isnull().sum())
data=df.dropna()
data.isnull().sum()
data

High          0
Low           0
Open          0
Close         0
Volume        0
Adj Close     0
EMA20        19
ROC20        19
RSI20        20
MA20         19
dtype: int64


High         Low  ...      RSI20        MA20
Ticker Date                                ...                       
TSLA   2019-01-31   62.312000   58.799999  ...  44.949086   63.782600
       2019-02-01   63.220001   60.700001  ...  45.282694   63.901100
       2019-02-04   63.060001   60.375999  ...  44.227442   63.853100
       2019-02-05   64.487999   62.450001  ...  45.591453   63.717000
       2019-02-06   64.848000   63.124001  ...  44.322238   63.535700
...                       ...         ...  ...        ...         ...
       2020-12-24  666.090027  641.000000  ...  92.405523  623.862003
       2020-12-28  681.400024  660.799988  ...  92.098041  627.758502
       2020-12-29  669.900024  655.000000  ...  91.794760  632.678003
       2020-12-30  696.599976  668.359985  ...  92.347409  638.179004
       2020-12-31  718.719971  691.119995  ...  92.312774  645.021503

[485 rows x 10 columns]